In [2]:
import pandas as pd

Creating the lists to be compared

The goal of the script is to create evenly sized lists of service providers which will ultimately be compared in a widely released publication.

The process_complaints function organises and aggregates the data by complaints publication and the services area for each year and creates the lists that will further be used to in a graphic as shown in the documents.

Where the complaints per 100k users is within 1 of the next service, it is added to the list. A new list is started for any service provider that isn't within 1 of the highest/lowest.

In [ ]:
def process_complaints(df, year, quarter, comparison_type):
    # Filter the DataFrame based on the specified year and quarter
    df_filtered = df[(df['Year'] == year) & (df['Quarter'] == quarter)]

    # Check if the filtered DataFrame is empty
    if df_filtered.empty:
        print(f"No data for Year {year} and Quarter {quarter}")
        return []
      
    # Initialize an empty list to store the result
    result = []

    # Group the filtered DataFrame by 'Service Area' and sort the groups
    for service_area, group in df_filtered.groupby('Service Area'):
        # Get the 'Complaints Publication CPs' and 'Complaints per 100000' values as lists
        cps = group['Complaints Publication CPs'].tolist()
        complaints_per_100000 = group['Complaints per 100000'].tolist()

        # Create a list to store comparisons for the current service area
        service_area_comparisons = []

        # Sort CPs by Complaints per 100000 in descending or ascending order based on comparison_type
        if comparison_type == 'Maximum':
            sorted_indices = sorted(range(len(cps)), key=lambda k: complaints_per_100000[k], reverse=True)
        elif comparison_type == 'Minimum':
            sorted_indices = sorted(range(len(cps)), key=lambda k: complaints_per_100000[k])

        sorted_cps = [cps[i] for i in sorted_indices]

        # Iterate over sorted CPs to find comparisons within 1 range
        for i in range(len(sorted_cps) - 1):
            cp = sorted_cps[i]
            cp_value = complaints_per_100000[sorted_indices[i]]

            # Create a list to store comparisons for the current CP
            cp_comparisons = []

            for j in range(i + 1, len(sorted_cps)):
                next_cp = sorted_cps[j]
                next_cp_value = complaints_per_100000[sorted_indices[j]]

                # Check if the absolute difference is within 1
                if (comparison_type == 'Maximum' and abs(next_cp_value - cp_value) <= 1) or \
                   (comparison_type == 'Minimum' and abs(next_cp_value - cp_value) <= 1):
                    cp_comparisons.append(next_cp)

            # Only add comparisons if they are within the specified range
            if cp_comparisons:
                service_area_comparisons.append([cp] + cp_comparisons)

        # Add service area comparisons to the result
        if service_area_comparisons:
            result.append((service_area, service_area_comparisons))

    return result

df = pd.read_excel("complaints_per_100000_update.xlsx") 

# Example for Maximum
result_max = process_complaints(df, 2023, 2, 'Maximum')

# Print the formatted results for Maximum
#print("Maximum Comparisons:")
#for service_area, comparisons in result_max:
#    print(f"{service_area}: {comparisons}")

# Example for Minimum
result_min = process_complaints(df, 2021, 2, 'Minimum')

# Print the formatted results for Minimum
#print("\nMinimum Comparisons:")
#for service_area, comparisons in result_min:
#    print(f"{service_area}: {comparisons}")


Filtering the lists / Removing duplication

the filter_list function here removes lists that contain an element found in any other list. This is so whenever a list is used in a comparison, it is never repeated.

In [6]:
def filter_lists(lists):
    i = 0
    while i < len(lists) - 1:
        if any(elem in lists[i] for elem in lists[i + 1]):
            del lists[i + 1]
            i = 0
        else:
            i += 1
    return lists

Grouping lists into serivce area

In [11]:
filtered_result_max = []
for service_area, comparisons in result_max:
    filtered_comparisons = filter_lists(comparisons)
    filtered_result_max.append((service_area, filtered_comparisons))

print("Maximum Comparisons after filtering:")
for service_area, comparisons in filtered_result_max:
    print(f"{service_area}: {comparisons}")

print("\n")
    
filtered_result_min = []
for service_area, comparisons in result_min:
    filtered_comparisons = filter_lists(comparisons)
    filtered_result_min.append((service_area, filtered_comparisons))

print("Minimum Comparisons after filtering:")
for service_area, comparisons in filtered_result_min:
    print(f"{service_area}: {comparisons}")

Maximum Comparisons after filtering:
Broadband: [['Lunar', 'Phoney'], ['Blossom', 'Rapid Broadband']]
Fixed Line: [['Phoney', 'Blossom', 'FFT', 'Lunar'], ['MOX', 'Rapid Broadband']]
Mobile - PAYG: [['Broadband 1', 'Loud Mobile', 'CO2', 'Union', 'Boost', 'MOX']]
Mobile Contract: [['Basic', 'Acme'], ['CO2', 'Boost']]


Minimum Comparisons after filtering:
Broadband: [['Basic', 'Acme'], ['Lunar', 'FFT']]
Fixed Line: [['Basic', 'Acme'], ['MoreSpeed', 'Lunar'], ['FFT', 'Rapid Broadband']]
Mobile - PAYG: [['Synergy', 'TopDawg', 'Union', 'CO2', 'Basic', 'Broadband 1', 'Boost'], ['Loud Mobile', 'MOX']]
Mobile Contract: [['Lunar', 'Basic', 'Acme', 'CO2'], ['Boost', 'MOX']]
